The Jacobian Transpose method is an inverse kinematic method that tries to find joint values to obtain an end effector position. The algorithm tries to approximate the inverse jacobian with the jacobian transpose to try and obtain a solution for the joint values.

In [95]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

In [96]:
# SOME HARD CODED VALUES FOR ROBOT PARAMETERS
z = 0.3
l1 = 0.5
l2 = 0.5
r1 = 0.2
r2 = 0.25
r3 = 0.3
phi = 1.3
theta1_value = 1.53
theta2_value = 1.51
theta3_value = 1.4
starting_angles = sym.Matrix([theta1_value, theta2_value, theta3_value])
theta1, theta2, theta3 = sym.symbols("theta1 theta2 theta3")


In [97]:
# MAKE A DH PARAMETER TABLE
# d, alpha, r, theta
dh_parameters = sym.Matrix(
  [
   [0, 0, 0, phi],
   [z + r1, 1.57, 0, 0],
   [0, 0, r1 + l1 + r2, theta1],
   [0, 0, r2 + l2 + r3, theta2],
   [0, 0, r3, theta3]
  ] 
)
print(dh_parameters)


Matrix([[0, 0, 0, 1.30000000000000], [0.500000000000000, 1.57000000000000, 0, 0], [0, 0, 0.950000000000000, theta1], [0, 0, 1.05000000000000, theta2], [0, 0, 0.300000000000000, theta3]])


In [98]:

# MAKE A DH MATRIX FOR THE ROBOT

# USING THE PARAMETERS 
dh_matrix = np.diag(np.ones((4)))

print(dh_parameters.shape[0])
for i in range(dh_parameters.shape[0]):
  
  trans_z = sym.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dh_parameters[i, 0]],
    [0, 0, 0, 1]
  ])
  dh_matrix = trans_z * dh_matrix
  rot_z = sym.Matrix([
    [sym.cos(dh_parameters[i, 3]), -1 * sym.sin(dh_parameters[i, 3]), 0, 0],
    [sym.sin(dh_parameters[i, 3]), sym.cos(dh_parameters[i, 3]), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0 ,1]
  ])
  dh_matrix =  rot_z * dh_matrix
  trans_x = sym.Matrix([
    [1, 0, 0, dh_parameters[i, 2]],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
  ])
  dh_matrix = trans_x * dh_matrix
  rot_x = sym.Matrix([
    [1, 0, 0, 0],
    [0, sym.cos(dh_parameters[i, 1]), -1 * sym.sin(dh_parameters[i, 1]), 0],
    [0, sym.sin(dh_parameters[i, 1]), sym.cos(dh_parameters[i, 1]), 0],
    [0, 0, 0, 1]
  ])
  dh_matrix = rot_x * dh_matrix
  
print(dh_matrix) 

5
Matrix([[-((-0.000767307120393385*sin(theta1) + 0.267498828624587*cos(theta1))*sin(theta2) + (0.267498828624587*sin(theta1) + 0.000767307120393385*cos(theta1))*cos(theta2))*sin(theta3) + ((-0.000767307120393385*sin(theta1) + 0.267498828624587*cos(theta1))*cos(theta2) - (0.267498828624587*sin(theta1) + 0.000767307120393385*cos(theta1))*sin(theta2))*cos(theta3), (-(-0.963558185417193*sin(theta1) + 0.000213016462323619*cos(theta1))*sin(theta2) + (-0.000213016462323619*sin(theta1) - 0.963558185417193*cos(theta1))*cos(theta2))*cos(theta3) - ((-0.963558185417193*sin(theta1) + 0.000213016462323619*cos(theta1))*cos(theta2) + (-0.000213016462323619*sin(theta1) - 0.963558185417193*cos(theta1))*sin(theta2))*sin(theta3), -(0.999999682931835*sin(theta1)*sin(theta2) - 0.999999682931835*cos(theta1)*cos(theta2))*sin(theta3) + (0.999999682931835*sin(theta1)*cos(theta2) + 0.999999682931835*sin(theta2)*cos(theta1))*cos(theta3), -((0.499999841465917*sin(theta1) + 0.95)*sin(theta2) - 0.499999841465917*co

In [99]:
# GET START POSITION
start_position = dh_matrix[0:3, -1]
print(start_position)

Matrix([[-((0.499999841465917*sin(theta1) + 0.95)*sin(theta2) - 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) + ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1) + 1.05)*cos(theta3) + 0.3], [((0.499999841465917*sin(theta1) + 0.95)*sin(theta2) - 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) + ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1) + 1.05)*sin(theta3)], [0.000398163355366632]])


In [100]:
# SET END POSITION


target_position = sym.Matrix([1, 1, 0.5])


In [101]:
# ALGORITHM

# CALCULATE THE JACOBIAN
angles = sym.Matrix([theta1, theta2, theta3])
# 
jacobian_matrix = start_position.jacobian(angles)

print(jacobian_matrix)

Matrix([[(-0.499999841465917*sin(theta1)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) + (-0.499999841465917*sin(theta1)*cos(theta2) - 0.499999841465917*sin(theta2)*cos(theta1))*sin(theta3), (-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) + (-(0.499999841465917*sin(theta1) + 0.95)*cos(theta2) - 0.499999841465917*sin(theta2)*cos(theta1))*sin(theta3), (-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) - ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1) + 1.05)*sin(theta3)], [(-0.499999841465917*sin(theta1)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) + (0.499999841465917*sin(theta1)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1))*cos(theta3), (-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) + ((0.499999841465

In [102]:

# CALCULATE THE JACOBIAN TRANSPOSE
jacobian_transpose = jacobian_matrix.transpose()

print(jacobian_transpose)

Matrix([[(-0.499999841465917*sin(theta1)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) + (-0.499999841465917*sin(theta1)*cos(theta2) - 0.499999841465917*sin(theta2)*cos(theta1))*sin(theta3), (-0.499999841465917*sin(theta1)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) + (0.499999841465917*sin(theta1)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1))*cos(theta3), 0], [(-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) + (-(0.499999841465917*sin(theta1) + 0.95)*cos(theta2) - 0.499999841465917*sin(theta2)*cos(theta1))*sin(theta3), (-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) + ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1))*cos(theta3), 0], [(-(0.499999841465917*sin(theta1) + 0.95)*sin(theta2) + 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) - ((0.49999984146

In [103]:
# GET A DIFFERENCE VECTOR

difference_position = target_position - start_position

print(difference_position)

Matrix([[((0.499999841465917*sin(theta1) + 0.95)*sin(theta2) - 0.499999841465917*cos(theta1)*cos(theta2))*sin(theta3) - ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1) + 1.05)*cos(theta3) + 0.7], [-((0.499999841465917*sin(theta1) + 0.95)*sin(theta2) - 0.499999841465917*cos(theta1)*cos(theta2))*cos(theta3) - ((0.499999841465917*sin(theta1) + 0.95)*cos(theta2) + 0.499999841465917*sin(theta2)*cos(theta1) + 1.05)*sin(theta3) + 1], [0.499601836644633]])


In [104]:
# GET JTe and JJTe

jte = jacobian_transpose * difference_position
jjte = jacobian_matrix * jte

In [105]:
# GET ALPHA

alpha = jte.dot(jte) / jjte.dot(jjte)

print(alpha.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value}))

0.266775033047401


In [108]:
# GET DELTA THETA

delta_theta_expr = alpha * jacobian_transpose * difference_position
delta_theta = delta_theta.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})


In [109]:
# GET NEW THETA

new_theta = starting_angles + delta_theta

print (new_theta)

Matrix([[1.51057694768575], [1.47397204615683], [0.813404393641391]])
